In [1]:
%load_ext autoreload
%autoreload 2
import torch
from src.networks.simple_unet import SUnetModel
from src.training.lightning_task import DenoiseRegressionTask
from src import config
from src.datasets.synthetic import SyntheticDataset, SyntheticDataModule
from monai.transforms import Compose, LoadImage, Orientation, ScaleIntensity
from lightning import Trainer
from lightning.pytorch.loggers import CometLogger

In [2]:
x = torch.rand((1, 1, *config.VOLUME_SHAPE))
model = DenoiseRegressionTask()

In [3]:
trainer = Trainer(
    max_epochs=10,
    logger=CometLogger(
        api_key=config.COMET_API_KEY,
        project_name="cortical",
        experiment_name="run-test",
    ),
    devices=2,
    strategy="ddp_notebook",
    accelerator="gpu",
    log_every_n_steps=200,
    precision="16-mixed",
)

CometLogger will be initialized in online mode
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, datamodule=SyntheticDataModule(6))

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/anaconda/envs/bowl/lib/python3.11/site-packages/torch/overrides.py:1739: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch_func_method(public_api, types, args, kwargs)
/opt/anaconda/envs/bowl/lib/python3.11/site-packages/torch/overrides.py:1739: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch_func_method(public_api, types, args, kwargs)


ProcessExitedException: process 1 terminated with signal SIGSEGV

In [6]:
model.model

SUnetModel(
  (encoder_level_1): SUnetBlock(
    (block): Sequential(
      (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): ReLU()
    )
  )
  (encoder_level_2): SUnetBlock(
    (block): Sequential(
      (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): ReLU()
    )
  )
  (encoder_level_3): SUnetBlock(
    (block): Sequential(
      (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
      (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, 